# Developing arXiv Tools

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import itertools as it
import magic
import os
import pprint
import shutil
import sys
import tempfile
import pandas as pd

sys.path.append('../..')
import tmx

In [3]:
# Folder of example files
example_dir = os.path.join(os.getcwd(), '../../data/arxiv/source/9912')
example_dir

# Number of files
len(os.listdir(example_dir))

2593

In [4]:
# Temporary directory for gzip archives
temp_dir = tempfile.mkdtemp(prefix='axv-')
print(temp_dir)

/var/folders/rp/zxyx9sp57n90yj92rrv4v3ph0000gn/T/axv-jgqxzrgh


In [5]:
# Empty the temporary directory ... needed for iterative work
for item in os.listdir(temp_dir):
    #print(f'Removing {item}')
    path = os.path.join(temp_dir, item)
    if os.path.isfile(path):
        os.remove(path)
    elif os.path.isdir(path):
        shutil.rmtree(path)
    else:
        raise OSError(f'not a file or directory: {item}')

## Copying and extracting gzipped archives

We need:

 1. a temporary directory
 2. a test for whether or not a file is gzipped
 3. a utility for ungzipping in Python (preferably)

In [6]:
example_items = map(
    lambda p: os.path.join(example_dir, p),
    os.listdir(example_dir)
)

ungzipped_paths = []
for i, file_path in enumerate(filter(os.path.isfile, example_items)):
    file_name = os.path.basename(file_path)
    file_id = os.path.splitext(file_name)[0]
    
    if not tmx.datasets.arxiv.is_gzipped(file_path):
        #print(f'Not gzipped: {file_name}')
        continue
    
    # Unpack the file: create a directory, copy it, and unpack it
    ungz_path = tmx.datasets.arxiv.stage_gzipped_file(file_path, temp_dir)
    ungzipped_paths += [ungz_path]
    #print(ungz_path)

In [7]:
len(ungzipped_paths)

2552

## Unpacking arXiv data

We have paths containing un-gzipped data. Now:

 1. Identify the file type
 2. Untar any tar files
 3. Identify any TeX files
 4. Return paths to any TeX files

Start by counting the kinds of files that we have available:

In [8]:
mime_counts = {}
for i, path in enumerate(ungzipped_paths):
    mime = magic.from_file(path, mime=True)
    if mime not in mime_counts:
        mime_counts[mime] = 0
    mime_counts[mime] += 1

pprint.pprint(mime_counts)

{'application/octet-stream': 13,
 'application/postscript': 16,
 'application/x-dvi': 1,
 'application/x-tar': 1747,
 'text/plain': 31,
 'text/x-Algol68': 1,
 'text/x-tex': 743}


In [9]:
text_files = []
for i, path in enumerate(ungzipped_paths):
    x = tmx.datasets.arxiv.gather_useable_text(
        *tmx.datasets.arxiv.collect_text_files(path)
    )
    text_files += [{
        'path': os.path.dirname(path),
        'files': x,
        'size': len(x)
    }]

In [10]:
df = pd.DataFrame(text_files)
df.shape

(2552, 3)

In [11]:
df.head()

files  \
0  [/var/folders/rp/zxyx9sp57n90yj92rrv4v3ph0000g...   
1  [/var/folders/rp/zxyx9sp57n90yj92rrv4v3ph0000g...   
2  [/var/folders/rp/zxyx9sp57n90yj92rrv4v3ph0000g...   
3  [/var/folders/rp/zxyx9sp57n90yj92rrv4v3ph0000g...   
4  [/var/folders/rp/zxyx9sp57n90yj92rrv4v3ph0000g...   

                                                path  size  
0  /var/folders/rp/zxyx9sp57n90yj92rrv4v3ph0000gn...     1  
1  /var/folders/rp/zxyx9sp57n90yj92rrv4v3ph0000gn...     1  
2  /var/folders/rp/zxyx9sp57n90yj92rrv4v3ph0000gn...     1  
3  /var/folders/rp/zxyx9sp57n90yj92rrv4v3ph0000gn...     1  
4  /var/folders/rp/zxyx9sp57n90yj92rrv4v3ph0000gn...     1

In [12]:
df_keep = df.loc[df['size'] > 0,:]
df_keep.shape

(2390, 3)

In [13]:
df_keep.head()

files  \
0  [/var/folders/rp/zxyx9sp57n90yj92rrv4v3ph0000g...   
1  [/var/folders/rp/zxyx9sp57n90yj92rrv4v3ph0000g...   
2  [/var/folders/rp/zxyx9sp57n90yj92rrv4v3ph0000g...   
3  [/var/folders/rp/zxyx9sp57n90yj92rrv4v3ph0000g...   
4  [/var/folders/rp/zxyx9sp57n90yj92rrv4v3ph0000g...   

                                                path  size  
0  /var/folders/rp/zxyx9sp57n90yj92rrv4v3ph0000gn...     1  
1  /var/folders/rp/zxyx9sp57n90yj92rrv4v3ph0000gn...     1  
2  /var/folders/rp/zxyx9sp57n90yj92rrv4v3ph0000gn...     1  
3  /var/folders/rp/zxyx9sp57n90yj92rrv4v3ph0000gn...     1  
4  /var/folders/rp/zxyx9sp57n90yj92rrv4v3ph0000gn...     1

In [14]:
# Missing text
df.shape[0]-df_keep.shape[0]

162

## Processing text

 1. Run through iconv to convert to UTF8
 1. Run through pandoc to remove macros
 2. Run through detex to remove latex

In [15]:
df_keep.iloc[0]['files']

['/var/folders/rp/zxyx9sp57n90yj92rrv4v3ph0000gn/T/axv-jgqxzrgh/astro-ph9912010/axv-untar/arcane.tex']

In [16]:
print(
    tmx.datasets.arxiv.detex(
        tmx.datasets.arxiv.pandoc(
            tmx.datasets.arxiv.iconv(source_path=df_keep.iloc[0]['files'][0]),
            template_path=os.path.join(os.getcwd(), '../../data/arxiv/pandoc-template.tex')
        )
    ).strip()
)

Limits on Arcminute Scale Cosmic Microwave Background Anisotropy with
the BIMA Array
W.L. Holzapfel, J.E. Carlstrom, L. Grego,

G. Holder, M. Joy, and E.D. Reese






W. L. Holzapfel 

Primary

To be submitted to Astrophysical Journal

r

1Department of Physics, University of California,
Berkeley CA 94720

2Department of Astronomy and Astrophysics, University of Chicago, Chicago
IL 60637

3Harvard-Smithsonian Center for Astrophysics, Mail Stop 83,
60 Gardeen St., Cambridge MA 02138

4Space Science Laboratory, SD50, NASA Marshall Space Flight Center,
Huntsville AL 35812

swlh@cfpa.berkeley.edu

cosmology: observation - cosmic microwave background

sec:intro
Introduction

The Cosmic Microwave Background (CMB) has the potential to be a powerful
probe of the early universe. In the standard inflationary model, the CMB
is imprinted with anisotropies that reflect the distribution of matter
at the epoch of recombination. Observations of the CMB at degree angular
scales probe structures which 